In [1]:
%pip install -qU torch transformers polars

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.4/36.4 MB 47.6 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
from pathlib import Path
import polars as pl
import numpy as np
import matplotlib.pyplot as plt
import torch
import json
import torch.nn.functional as F
from torch import Tensor
from tqdm import tqdm, trange
import pickle
from transformers import AutoTokenizer, AutoModel

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

## Load data

### Summaries

In [28]:
SUMMARIES_FOLDER = Path("/content/drive/MyDrive/Telegram Marketing/Embeddings/channels/not_simple/ad_prev_summaries/")

summaries = {}
for batch_path in tqdm(Path(SUMMARIES_FOLDER).iterdir()):
    with open(batch_path, "r") as f:
        summaries.update(json.load(f))

len(summaries)

62it [00:26,  2.33it/s]


310

In [29]:
next(iter(summaries.items()))

('3857852',
 '1. Тематика канала: финансы, инвестиции, фондовый рынок.\n2. Тональность контента: формальная, образовательная, аналитическая.\n3. Целевая аудитория: взрослые люди, преимущественно мужчины, интересующиеся инвестициями и фондовым рынком, финансовые аналитики и трейдеры.\n4. Общий стиль подачи информации: аналитические и информативные посты, содержащие данные и прогнозы, короткие и ёмкие сообщения с актуальной информацией.')

### Channel description

In [30]:
ch_descr_path = Path("/content/drive/MyDrive/Telegram Marketing/Embeddings/channels/channels_data_for_emb.csv")
ch_descr = pl.read_csv(ch_descr_path)
ch_descr.drop_in_place("last_pinned_msg_id")
ch_descr

id,title,about
i64,str,str
1097289882,"""Medical Ксю""","""Канал о цифровых технологиях в…"
1203719412,"""Design Porn""",""""""
1336205252,"""Tolmaç Channel""","""Group of @TolmacBot Chat @tolm…"
1906656409,"""Gaimin Gladiators""",""""""
1785605454,"""Антон Архипов | Про инфобиз""","""Про проекты и работу команды. …"
…,…,…
1667226149,"""🐈Calico Cat🐈‍⬛""","""Мяукаю"""
2223746475,"""аушня от малого❄⛄""","""🦔🍂🦔 анон - @navimmilBot с вонк…"
1447537721,"""Sun above my head|| Кеметизм""","""+18, рекламы нет Под солнцем в…"


### Ad posts metadata

In [31]:
ad_posts_meta = pl.read_csv("/content/drive/MyDrive/Telegram Marketing/Embeddings/ad_posts/ad_posts_meta.csv")
ad_posts_meta.drop_in_place("post_date")
ad_posts_meta

id,channel_id
i64,i64
6901405,1229173666
6901333,1229173666
6900679,1229173666
6900562,1229173666
6899803,1229173666
…,…
6902253,1229173666
6902226,1229173666
6902013,1229173666


## Construct text for embeddings

In [35]:
stats = {"ad_posts": len(ad_posts_meta), "about": 0, "summary": 0}
documents = []

for ad_post_id, channel_id in tqdm(ad_posts_meta.iter_rows()):
    _, title, about = ch_descr.row(by_predicate=(pl.col("id") == channel_id))
    summary = summaries.get(str(ad_post_id), None)

    doc = f"Название: {title}"
    if about:
        stats["about"] += 1
        doc += f"\nОписание: {about}"
    if summary:
        stats["summary"] += 1
        doc += f"\nХарактеристика контента: {summary}"

    documents.append(doc)

177373it [01:02, 2834.48it/s]


In [36]:
# Analyze stats
print(f"""ad posts = {stats['ad_posts']}, 100%
about texts = {stats['about']}, {stats['about']/stats['ad_posts']*100}% of ad posts
prev posts summaries = {stats['summary']}, {stats['summary']/stats['ad_posts']*100}% of ad posts""")

ad posts = 177373, 100%
about texts = 176042, 99.24960394197538% of ad posts
prev posts summaries = 310, 0.1747729361289486% of ad posts


In [41]:
# Print doc with prev posts summaries
print(documents[ad_posts_meta.with_row_index().row(by_predicate=(pl.col("id") == 3857852))[0]])

Название: БКС Экспресс
Описание: Официальный канал издания БКС Экспресс. Все об инвестициях: новости, аналитика, идеи и рекомендации. 

💻 Сайт: bcs-express.ru 

📱 Мобильное приложение БКС: https://s.bcs.ru/8mwl 

✉️ Для связи: ifs@bcs.ru
Характеристика контента: 1. Тематика канала: финансы, инвестиции, фондовый рынок.
2. Тональность контента: формальная, образовательная, аналитическая.
3. Целевая аудитория: взрослые люди, преимущественно мужчины, интересующиеся инвестициями и фондовым рынком, финансовые аналитики и трейдеры.
4. Общий стиль подачи информации: аналитические и информативные посты, содержащие данные и прогнозы, короткие и ёмкие сообщения с актуальной информацией.


In [42]:
def get_detailed_instruct(task_description: str, query: str) -> str:
    return f'Задача: {task_description}\nДанные: {query}'

In [43]:
task = "Проанализировав данные Telegram-канала (название, описание, характеристика контента), опиши его типичный профиль (тематика, аудитория, тон общения)"

input_texts = []
for doc in tqdm(documents):
    input_texts.append(get_detailed_instruct(task, doc))

100%|██████████| 177373/177373 [00:00<00:00, 693811.09it/s]


In [44]:
print(input_texts[ad_posts_meta.with_row_index().row(by_predicate=(pl.col("id") == 3857852))[0]])

Задача: Проанализировав данные Telegram-канала (название, описание, характеристика контента), опиши его типичный профиль (тематика, аудитория, тон общения)
Данные: Название: БКС Экспресс
Описание: Официальный канал издания БКС Экспресс. Все об инвестициях: новости, аналитика, идеи и рекомендации. 

💻 Сайт: bcs-express.ru 

📱 Мобильное приложение БКС: https://s.bcs.ru/8mwl 

✉️ Для связи: ifs@bcs.ru
Характеристика контента: 1. Тематика канала: финансы, инвестиции, фондовый рынок.
2. Тональность контента: формальная, образовательная, аналитическая.
3. Целевая аудитория: взрослые люди, преимущественно мужчины, интересующиеся инвестициями и фондовым рынком, финансовые аналитики и трейдеры.
4. Общий стиль подачи информации: аналитические и информативные посты, содержащие данные и прогнозы, короткие и ёмкие сообщения с актуальной информацией.


## Analyze length

In [ ]:
pl.DataFrame([len(i) for i in input_texts]).describe()

In [ ]:
plt.hist([len(i) for i in input_texts], bins=50);

In [ ]:
# Analyze tokenized length
model_name = 'intfloat/multilingual-e5-large-instruct'
tokenizer = AutoTokenizer.from_pretrained(model_name)

tok_length = []
for inp in tqdm(input_texts):
    tokens = tokenizer(inp, padding=False, truncation=False, return_tensors="np", verbose=True)['input_ids']
    tok_length.append(tokens.shape[1])

In [ ]:
plt.hist(tok_length, bins=50);

In [ ]:
pl.DataFrame(tok_length).describe()

In [ ]:
len([l for l in tok_length if l > 514]) / len(tok_length) * 100

## Calc embeddings

In [ ]:
model_name = 'intfloat/multilingual-e5-large-instruct'

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)
model = model.to(device)

In [ ]:
def average_pool(last_hidden_states: Tensor,
                 attention_mask: Tensor) -> Tensor:
    last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
    return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]


def process_in_batches(model, tokenizer, input_texts, batch_size=16, device=torch.device("cpu")):
    embeddings = []
    for i in trange(0, len(input_texts), batch_size):
        batch_texts = input_texts[i:i + batch_size]

        batch_dict = tokenizer(batch_texts, max_length=512, padding=True, truncation=True, return_tensors='pt')
        batch_dict = {k: v.to(device) for k, v in batch_dict.items()}

        with torch.no_grad():
            outputs = model(**batch_dict)
            batch_embeddings = average_pool(outputs.last_hidden_state, batch_dict['attention_mask'])

        batch_embeddings = F.normalize(batch_embeddings, p=2, dim=1)
        embeddings.append(batch_embeddings.cpu())

        del batch_dict, outputs, batch_embeddings
        torch.cuda.empty_cache()


    return torch.cat(embeddings, dim=0)


In [ ]:
embeddings = process_in_batches(model, tokenizer, input_texts, batch_size=16, device=device)
embeddings.shape

In [ ]:
# Save embeddings
EMB_FILE = Path("/content/drive/MyDrive/Telegram Marketing/Embeddings/channels/not_simple/ch_sum_emb_e5_instruct.npy")
np.save(EMB_FILE, embeddings.numpy())

# Also save indices of posts
IDS_FILE = Path("/content/drive/MyDrive/Telegram Marketing/Embeddings/channels/not_simple/ids_ch_sum_emb_e5_instruct.npy")
np.save(IDS_FILE, ad_posts_meta["id"].to_numpy())

In [ ]:
print(torch.cuda.memory_summary())